In [1]:
import pandas as pd
import openpyxl
import numpy as np
import os
import glob2
import glob
import random
import sklearn
import ntpath
import json
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
from nltk.test.portuguese_en_fixt import setup_module
import nltk
from contextlib import nullcontext


In [2]:
## Classe para criação de dataframes 
class read_file():
    def __init__(self, file_name):
        self.file_name = file_name
        self.df = pd.read_excel(self.file_name,engine='openpyxl')

    def get_df(self):
        return self.df

    def get_file_name(self):
        return self.file_name
    
    def clean_df(self):
        self.df.dropna(inplace=True)
        self.df.drop_duplicates(inplace=True)
        self.df.reset_index(drop=True, inplace=True)
        return self.df
    
    def get_df_columns(self):
        return self.df.columns
    

    def get_file_path(self):
        return os.path.dirname(self.file_name)

    def get_file_name_without_extension(self):
        return ntpath.basename(self.file_name).split('.')[0]

    def get_file_name_with_extension(self):
        return ntpath.basename(self.file_name)

    def get_file_name_extension(self):
        return ntpath.basename(self.file_name).split('.')[1]

    def get_file_name_without_extension_and_path(self):
        return ntpath.basename(self.file_name).split('.')[0]

    def get_file_name_with_extension_and_path(self):
        return self.file_name

    def get_file_name_extension_and_path(self):
        return ntpath.basename(self.file_name).split('.')[1]

    def get_file_name_without_extension_and_path_and_separator(self):
        return ntpath.basename(self.file_name).split('.')[0].replace('\\', '/')

    def get_file_name_with_extension_and_path_and_separator(self):
        return self.file_name.replace('\\', '/')

    def get_file_name_extension_and_path_and_separator(self):
        return ntpath.basename(self.file_name).split('.')[1].replace('\\', '/')

    def get_file_name_without_extension_and_path_and_separator_and_dot(self):
        return ntpath



In [3]:
#Função para listar campos nulos de um dataframe e remover as colunas puramente nulas

def list_columns_and_remove_nullcontext(dataframe):
    null_columns= []
    for column in dataframe.columns:
        if dataframe[column].isnull().all():
            null_columns = null_columns + [column]
    print(null_columns)
    dataframe.drop(null_columns, axis=1, inplace=True)

#### Criação dos dataframes do modelo

In [4]:
dataframe_inventario = read_file('Files/Inventario.xlsx').get_df() # Cria dataframe de inventário
dataframe_transacoes_produto = read_file('Files/Transações_por_produto.xlsx').get_df() # Cria dataframe de transações por produto
dataframe_transacoes_vendas = read_file('Files/Transações_vendas.xlsx').get_df() # Cria dataframe de transações por venda

#### Checando o dataframe de transação

In [5]:
dataframe_transacoes_produto.head(15)

,NF,Ent,Tipo,Número,Data,Hora,Cliente / Fornecedor,Produto,Quant.,Vl. Unitário,Sub-Total,Desconto,Outras Despesas,Total,Funcionário,Valor Comissão,Comissão %,Calc.Comissão
0,NaN,NaN,Venda,38253.0,2022-07-30 11:45:35.193,2022-07-30 11:45:35.193,NaN,TECIDO TRICOLINE DIVERSOS,-0.25,27.90,6.97,0.00,0,6.97,Administrador,0,0,Sob.Total
1,NaN,NaN,Venda,38252.0,2022-07-30 09:45:20.596,2022-07-30 09:45:20.596,NaN,FITA CETIM PROGRESSO 0 -209 VERMELHO,-100.00,0.35,35.00,7.00,0,28.00,Administrador,0,0,Sob.Total
2,NaN,NaN,Venda,38251.0,2022-07-30 09:36:37.140,2022-07-30 09:36:37.140,NaN,"FITA MÉTRICA NYBC 1,5m",-1.00,5.80,5.80,0.00,0,5.80,Administrador,0,0,Sob.Total
3,NaN,NaN,NaN,NaN,NaT,NaT,NaN,COSTURA DRIMA - Branca,-1.00,3.00,3.00,0.00,0,3.00,Administrador,0,0,Sob.Total
4,NaN,NaN,NaN,NaN,NaT,NaT,NaN,COSTURA DRIMA - 5605,-1.00,3.00,3.00,0.00,0,3.00,Administrador,0,0,Sob.Total
5,NaN,NaN,Venda,38248.0,2022-07-29 17:29:13.607,2022-07-29 17:29:13.607,NaN,MINIATURA LASER CADEIRA,-2.00,4.80,9.60,0.00,0,9.60,Administrador,0,0,Sob.Total
6,NaN,NaN,NaN,NaN,NaT,NaT,NaN,COLA TEK BOND 2 20G,-1.00,16.80,16.80,0.00,0,16.80,Administrador,0,0,Sob.Total
7,NaN,NaN,Venda,38247.0,2022-07-29 16:37:15.597,2022-07-29 16:37:15.597,NaN,AGULHA COSTURA -Darning MILWARD 5,-3.00,1.00,3.00,0.00,0,3.00,Administrador,0,0,Sob.Total
8,NaN,NaN,NaN,NaN,NaT,NaT,NaN,AGULHA COSTURA -Darning MILWARD 3,-2.00,1.00,2.00,0.00,0,2.00,Administrador,0,0,Sob.Total
9,NaN,NaN,NaN,NaN,NaT,NaT,NaN,AGULHA COSTURA -Darning N 1,-2.00,1.25,2.50,0.00,0,2.50,Administrador,0,0,Sob.Total


Como podemos ver, neste caso tivemos o arquivo agrupado no excel. O mesmo agrupamento não foi mantido na leitura, assim sendo necessário além da remoção de colunas nulas, uma função em que replique o último valor por cada tranação para que complete os dados que competem à mesma transação.

#### Checando o dataframe de inventário

In [6]:
dataframe_inventario.head()

,Descrição,Código,Estoque Atual,Estoque Reservado,Estoque Disponível,Preço,Categoria,Código Extra,Unidade,Fornecedor Principal,...,Comissão,Localização,Promoção,Data Inicial,Data Final,Preço Promocional,Status,Kit / Combo,EAN / GTIN,Exportar para balança
0,APLIQUE - APM4 -053 NOIVOS,7100692,9.0,0.0,9.0,2.5,PAPELARIA,NaN,NaN,NaN,...,NaN,NaN,NAO,NaT,NaT,NaN,Ativo,Não,7.893798e+12,Não
1,BARROCO MULT 200G -9384 GREENNERY,099810,0.0,0.0,0.0,22.8,Linha,NaN,NaN,NaN,...,NaN,NaN,NAO,NaT,NaT,NaN,Ativo,Não,7.891114e+12,Não
2,ABAJUR MARINHEIRO,124205,1.0,0.0,1.0,38.0,MDF,NaN,NaN,NaN,...,NaN,NaN,NAO,NaT,NaT,NaN,Ativo,Não,NaN,Não
3,ABAJUR MDF DECORADO BEBE URSA,8099512,1.0,0.0,1.0,65.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NAO,NaT,NaT,NaN,Ativo,Não,NaN,Não
4,ABRIDOR DE CASA G,7101086,2.0,0.0,2.0,5.0,ARMARINHO,NaN,NaN,NaN,...,NaN,NaN,NAO,NaT,NaT,NaN,Ativo,Não,NaN,Não


Para os arquivos de inventário, é possível remover colunas puramente nulas, pois também, como no dataframe anterior, não agregarão ao modelo.

#### Feature engineering:


### Dataframe de Transações por produto

Neste caso, é interessante o uso do método de beforefill do pandas par que o registro posterior receba o valor do anterior, assim ajustando os agrupamentos do arquivo xlsx.

In [7]:
dataframe_transacoes_produto['Tipo'] = dataframe_transacoes_produto['Tipo'].fillna(method='bfill')
dataframe_transacoes_produto['Número'] = dataframe_transacoes_produto['Número'].fillna(method='bfill')
dataframe_transacoes_produto['Data'] = dataframe_transacoes_produto['Data'].fillna(method='bfill')
dataframe_transacoes_produto['Hora'] = dataframe_transacoes_produto['Hora'].fillna(method='bfill')

In [8]:
dataframe_transacoes_produto_tratado = dataframe_transacoes_produto
dataframe_transacoes_produto_tratado.head()

,NF,Ent,Tipo,Número,Data,Hora,Cliente / Fornecedor,Produto,Quant.,Vl. Unitário,Sub-Total,Desconto,Outras Despesas,Total,Funcionário,Valor Comissão,Comissão %,Calc.Comissão
0,NaN,NaN,Venda,38253.0,2022-07-30 11:45:35.193,2022-07-30 11:45:35.193,NaN,TECIDO TRICOLINE DIVERSOS,-0.25,27.90,6.97,0.0,0,6.97,Administrador,0,0,Sob.Total
1,NaN,NaN,Venda,38252.0,2022-07-30 09:45:20.596,2022-07-30 09:45:20.596,NaN,FITA CETIM PROGRESSO 0 -209 VERMELHO,-100.00,0.35,35.00,7.0,0,28.00,Administrador,0,0,Sob.Total
2,NaN,NaN,Venda,38251.0,2022-07-30 09:36:37.140,2022-07-30 09:36:37.140,NaN,"FITA MÉTRICA NYBC 1,5m",-1.00,5.80,5.80,0.0,0,5.80,Administrador,0,0,Sob.Total
3,NaN,NaN,Venda,38248.0,2022-07-29 17:29:13.607,2022-07-29 17:29:13.607,NaN,COSTURA DRIMA - Branca,-1.00,3.00,3.00,0.0,0,3.00,Administrador,0,0,Sob.Total
4,NaN,NaN,Venda,38248.0,2022-07-29 17:29:13.607,2022-07-29 17:29:13.607,NaN,COSTURA DRIMA - 5605,-1.00,3.00,3.00,0.0,0,3.00,Administrador,0,0,Sob.Total


Neste passo para uma boa análise exploratória dos dados, é interessante remover colunas desinteressantes para o modelo. A função criada no início do modelo pode ser usada para a remoção de campos nulos de cada dataframe.

In [9]:
list_columns_and_remove_nullcontext(dataframe_transacoes_produto_tratado)
dataframe_transacoes_produto_tratado.head()

['NF', 'Ent']


,Tipo,Número,Data,Hora,Cliente / Fornecedor,Produto,Quant.,Vl. Unitário,Sub-Total,Desconto,Outras Despesas,Total,Funcionário,Valor Comissão,Comissão %,Calc.Comissão
0,Venda,38253.0,2022-07-30 11:45:35.193,2022-07-30 11:45:35.193,NaN,TECIDO TRICOLINE DIVERSOS,-0.25,27.90,6.97,0.0,0,6.97,Administrador,0,0,Sob.Total
1,Venda,38252.0,2022-07-30 09:45:20.596,2022-07-30 09:45:20.596,NaN,FITA CETIM PROGRESSO 0 -209 VERMELHO,-100.00,0.35,35.00,7.0,0,28.00,Administrador,0,0,Sob.Total
2,Venda,38251.0,2022-07-30 09:36:37.140,2022-07-30 09:36:37.140,NaN,"FITA MÉTRICA NYBC 1,5m",-1.00,5.80,5.80,0.0,0,5.80,Administrador,0,0,Sob.Total
3,Venda,38248.0,2022-07-29 17:29:13.607,2022-07-29 17:29:13.607,NaN,COSTURA DRIMA - Branca,-1.00,3.00,3.00,0.0,0,3.00,Administrador,0,0,Sob.Total
4,Venda,38248.0,2022-07-29 17:29:13.607,2022-07-29 17:29:13.607,NaN,COSTURA DRIMA - 5605,-1.00,3.00,3.00,0.0,0,3.00,Administrador,0,0,Sob.Total


Feito isso, um breve perfilamento dos dados trará informações para continuidade da limpeza dos dados, assim habilitando as explorações:

In [13]:
dataframe_transacoes_produto_tratado.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
Número,47175.0,20458.603922,10843.825850,1.00,11753.5,21035.0,29809.000,38253.0
Quant.,47175.0,2.250551,125.751798,-10092.06,-1.0,-1.0,0.000,11988.0
Vl. Unitário,47175.0,6.850450,17.851295,0.00,0.0,2.5,9.375,1174.0
Sub-Total,47175.0,5.838035,14.485489,0.00,0.0,0.0,6.000,700.0
Desconto,47175.0,0.159503,1.440226,0.00,0.0,0.0,0.000,98.0
Outras Despesas,47175.0,0.000000,0.000000,0.00,0.0,0.0,0.000,0.0
Total,47175.0,5.678532,13.948205,0.00,0.0,0.0,6.000,700.0
Valor Comissão,47175.0,0.000000,0.000000,0.00,0.0,0.0,0.000,0.0
Comissão %,47175.0,0.000000,0.000000,0.00,0.0,0.0,0.000,0.0



### Dataframe de Inventário

O mesmo tratamento de remoção de colunas puramente nulas será feito para o dataframe de inventário.

In [10]:
list_columns_and_remove_nullcontext(dataframe_inventario)
dataframe_inventario.head()

['Código Extra', 'Código Ref.', 'Pontos Nec.', 'Tributação', 'NCM', 'CEST', 'Peso Líq.', 'Peso Bruto', 'Comissão', 'Localização']


,Descrição,Código,Estoque Atual,Estoque Reservado,Estoque Disponível,Preço,Categoria,Unidade,Fornecedor Principal,Marca,...,Alterado em,Alterado por,Promoção,Data Inicial,Data Final,Preço Promocional,Status,Kit / Combo,EAN / GTIN,Exportar para balança
0,APLIQUE - APM4 -053 NOIVOS,7100692,9.0,0.0,9.0,2.5,PAPELARIA,NaN,NaN,LITOARTE,...,2021-04-13 15:15:27.708,admin,NAO,NaT,NaT,NaN,Ativo,Não,7.893798e+12,Não
1,BARROCO MULT 200G -9384 GREENNERY,099810,0.0,0.0,0.0,22.8,Linha,NaN,NaN,Círculo,...,2022-05-07 07:56:56.014,admin,NAO,NaT,NaT,NaN,Ativo,Não,7.891114e+12,Não
2,ABAJUR MARINHEIRO,124205,1.0,0.0,1.0,38.0,MDF,NaN,NaN,NaN,...,NaT,NaN,NAO,NaT,NaT,NaN,Ativo,Não,NaN,Não
3,ABAJUR MDF DECORADO BEBE URSA,8099512,1.0,0.0,1.0,65.0,NaN,NaN,NaN,NaN,...,2022-02-08 09:41:08.406,admin,NAO,NaT,NaT,NaN,Ativo,Não,NaN,Não
4,ABRIDOR DE CASA G,7101086,2.0,0.0,2.0,5.0,ARMARINHO,NaN,NaN,NaN,...,2021-10-11 09:35:33.641,admin,NAO,NaT,NaT,NaN,Ativo,Não,NaN,Não
